In [2]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
url = 'https://raw.githubusercontent.com/vijaycse/Amazon_Vine_Analysis/master/amazon_reviews_public_vine_table.csv'
ratings = pd.read_csv(url, index_col=0)

In [4]:
ratings.head(10)

,star_rating,helpful_votes,total_votes,vine,verified_purchase
review_id,,,,,
R26MV8D0KG6QI6,5.0,0.0,0.0,N,Y
R1OF8GP57AQ1A0,5.0,0.0,0.0,N,Y
R3VDC1QB6MC4ZZ,5.0,0.0,0.0,N,N
R12FA3DCF8F9ER,5.0,0.0,0.0,N,Y
RTWHVNV6X4CNJ,5.0,0.0,0.0,N,Y
RIG9AWFOGRDVO,2.0,1.0,1.0,N,Y
R1S1XSG4ZCHDGS,5.0,1.0,1.0,N,Y
RB15NBVY5ELVW,5.0,2.0,2.0,N,Y
R56358YM1ZJ7I,5.0,0.0,0.0,N,N


In [10]:
# filter DataFrame for total_votes above or equal to 20
df1 = ratings[(ratings["total_votes"] >= 20)]
df1

,star_rating,helpful_votes,total_votes,vine,verified_purchase
review_id,,,,,
R1OAZUG90XPU2U,5.0,250.0,274.0,N,Y
R23VPTUJ8I8NR5,1.0,28.0,30.0,N,Y
R33VIWRD2X5IA9,1.0,0.0,20.0,N,Y
R6SRJFJ5YH4UM,5.0,23.0,26.0,N,Y
R3DOS07BIMP4DJ,2.0,32.0,42.0,N,N
...,...,...,...,...,...
R1IQNIT2UFQE97,5.0,26.0,27.0,N,N
RIQSTVHD4MQHH,5.0,21.0,23.0,N,N
RZOSXUI2V2LO7,5.0,23.0,23.0,N,N


In [12]:
# Filter DataFrame for helpful_votes ratio above or equal to 50%
df2 = ratings[((ratings["helpful_votes"] / ratings["total_votes"]) >= 0.5)]
df2.head()

,star_rating,helpful_votes,total_votes,vine,verified_purchase
review_id,,,,,
RIG9AWFOGRDVO,2.0,1.0,1.0,N,Y
R1S1XSG4ZCHDGS,5.0,1.0,1.0,N,Y
RB15NBVY5ELVW,5.0,2.0,2.0,N,Y
R155ZMVLD5C9BP,4.0,3.0,4.0,N,Y
R2IXW43PUNYM0G,4.0,1.0,1.0,N,Y


In [14]:
# Create paid vine DataFrame
paid_df = ratings[(ratings["vine"] =='Y')]
paid_df.head(5)

,star_rating,helpful_votes,total_votes,vine,verified_purchase
review_id,,,,,
R74RZQRH3MKUW,3.0,0.0,0.0,Y,N
R37HYU6L7KWNSR,5.0,0.0,0.0,Y,N
R10ZYDTCBALOLE,2.0,1.0,1.0,Y,N
R2SG1ZRKMJIMBE,4.0,0.0,0.0,Y,N
R1R0XMH6L7GZ2N,5.0,0.0,0.0,Y,N


In [37]:
# Create unpaid vine DataFrame
unpaid_df = ratings[(ratings["vine"] =='N')]
unpaid_df.head(5)

,star_rating,helpful_votes,total_votes,vine,verified_purchase
review_id,,,,,
R26MV8D0KG6QI6,5.0,0.0,0.0,N,Y
R1OF8GP57AQ1A0,5.0,0.0,0.0,N,Y
R3VDC1QB6MC4ZZ,5.0,0.0,0.0,N,N
R12FA3DCF8F9ER,5.0,0.0,0.0,N,Y
RTWHVNV6X4CNJ,5.0,0.0,0.0,N,Y


In [39]:
# paid total number of reviews
#paid_df.count
total_paid_reviews = paid_df.count()
total_paid_reviews

star_rating          16612
helpful_votes        16612
total_votes          16612
vine                 16612
verified_purchase    16612
dtype: int64

In [40]:
# paid total number of reviews
#paid_df.count
total_unpaid_reviews = unpaid_df.count()
total_unpaid_reviews

star_rating          2385819
helpful_votes        2385819
total_votes          2385819
vine                 2385819
verified_purchase    2385819
dtype: int64

In [44]:
# paid 5-star reviews
paid_five_star_reviews_df = paid_df[paid_df["star_rating"] == 5.0]
paid_five_star_reviews_df

total_five_star_paid_reviews = paid_five_star_reviews_df.count() 
total_five_star_paid_reviews

star_rating          5657
helpful_votes        5657
total_votes          5657
vine                 5657
verified_purchase    5657
dtype: int64

In [43]:
# paid 5-star reviews percentage
paid_five_star_percent = (total_five_star_paid_reviews / total_paid_reviews) * 100
paid_five_star_percent

star_rating          10006.308693
helpful_votes        10006.308693
total_votes          10006.308693
vine                 10006.308693
verified_purchase    10006.308693
dtype: float64

In [47]:
# unpaid 5-star reviews
unpaid_five_star_reviews_df = unpaid_df[unpaid_df["star_rating"] == 5.0]
unpaid_five_star_reviews_df

total_five_star_unpaid_reviews = unpaid_five_star_reviews_df.count() 
total_five_star_unpaid_reviews

star_rating          1656591
helpful_votes        1656591
total_votes          1656591
vine                 1656591
verified_purchase    1656591
dtype: int64

In [48]:
# unpaid 5-star reviews percentage
unpaid_five_star_percent = (total_five_star_unpaid_reviews / total_unpaid_reviews) * 100
unpaid_five_star_percent

star_rating          69.434898
helpful_votes        69.434898
total_votes          69.434898
vine                 69.434898
verified_purchase    69.434898
dtype: float64